# NEXUS tool: case study for the NWSAS basin - least-cost options calculations
In this notebook a case study for the NWSAS basin is covered using the `nexus_tool` package. The energy requirements for agricultural irrigation were previously calculated (check the water demand and energy demand case studies). In this case study, the least-cost options to supply the energy needs for agricultural irrigation are identified.

First import the package by running the following block:

In [107]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
%autoreload
import nexus_tool

[autoreload of nexus_tool failed: Traceback (most recent call last):
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 387, in update_generic
    update(a, b)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 342, in update_class
    if update_generic(old_obj, new_obj): continue
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 387, in update_generic
    update(a, b)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packa

In [109]:
file_path = r'nwsas_results/pumping_energy.csv'
df = nexus_tool.read_csv(file_path)

In [110]:
file_path = r'nwsas_data/Fuel_prices.xlsx'
fuel_price = nexus_tool.read_excel(file_path)

In [111]:
nwsas = nexus_tool.Model(df)
nwsas.end_year = 2030
nwsas.start_year = 2016
nwsas.discount_rate = 0.05

In [112]:
nwsas.df = nwsas.df.merge(fuel_price, on='country')

In [113]:
nwsas.create_wind_turbine('Wind power', life=20,
                          om_cost=0.02, capital_cost=1300,
                          efficiency=0.6)

In [114]:
nwsas.create_pv_system('Solar PV', life=15,
                       om_cost=0.01, capital_cost=1140,
                       efficiency=0.7)

In [115]:
nwsas.create_standard_tech('Diesel set', life=10, om_cost=0.1,
                           capital_cost=938, fuel_cost=nwsas.df['Dprice1'],
                           fuel_req=0.095, efficiency=0.27, cf = 0.5,
                           emission_factor=2.7, env_cost=0)

In [116]:
nwsas.create_standard_tech('Grid pump', life=10, om_cost=0.1,
                           capital_cost=845, fuel_cost=nwsas.df['Epricelow'],
                           fuel_req=1, efficiency=0.55, cf = 0.8,
                           emission_factor=0.728, env_cost=0)

In [122]:
nwsas.get_cf(['Wind power', 'Solar PV'])
nwsas.get_installed_capacity('all')
nwsas.get_max_capacity('a')
nwsas.get_lcoe()

In [123]:
nwsas.df['wind_lcoe'] = nwsas.technologies['Wind power'].df['lcoe']
nwsas.df['pv_lcoe'] = nwsas.technologies['Solar PV'].df['lcoe']
nwsas.df['diesel_lcoe'] = nwsas.technologies['Diesel set'].df['lcoe']
nwsas.df['grid_lcoe'] = nwsas.technologies['Grid pump'].df['lcoe']
nwsas.df[['province','wind_lcoe','pv_lcoe', 'diesel_lcoe', 'grid_lcoe']].groupby('province').mean()

,wind_lcoe,pv_lcoe,diesel_lcoe,grid_lcoe
province,,,,
Adrar,10.359339,0.770447,0.612801,0.362259
Biskra,22.287251,0.950498,0.565520,0.335638
Djelfa,12.942626,0.897963,0.568539,0.337337
El Oued,18.697837,0.910149,0.601037,0.355635
Gabes,15.892456,0.940353,0.766064,0.490313
Ghadamis,11.332625,0.845636,0.559607,0.598376
Ghardaia,8.050849,0.814292,0.593385,0.351327
Gharyan,20.290699,1.140647,0.945389,0.815585
Illizi,10.186315,0.759492,0.597699,0.353756
